In [11]:
import numpy as np

# Matrix Factorization to predict user ratings

Ratings matrix R is of size U x D (U = number of users, D = number of items to be rated). It is a sparse matrix, as it contains 0 values where the user has not seen/rated the film.


The goal is to discover K latent features, and also predict what users might rate the films they haven't rated/seen, as part of a recommendation system.


Given two input matrix P and Q, these are used to generate a predicted R which will include predicted ratings for films Users haven't rated yet, based on their ratings for other films and how similar users have rated. 

The predicted R is given by P * QT    (QT = transpose of Q)


Matrix P describes the relationship between Users (U) and the latent features (K), whilst Q describes the relationship between the Items (D) and the latent features (K).

To get P and Q, we initially use two random matrices and calculate the difference of the product named as matrix M. Next, we minimize the difference through the iterations via gradient descent, aiming at finding a local minimum of the difference.

In [4]:
# R = ratings matrix, 1 row per user, 1 column per film
R = np.array([[5,3,0,1],[4,0,0,1],[1,1,0,5],[1,0,0,4],[0,1,5,4],[2,1,3,0]])
R

array([[5, 3, 0, 1],
       [4, 0, 0, 1],
       [1, 1, 0, 5],
       [1, 0, 0, 4],
       [0, 1, 5, 4],
       [2, 1, 3, 0]])

In [12]:
# N: num of users
N = len(R)
# M: num of movies
M = len(R[0])
# Num of latent features we wish to extract
K = 3

In [9]:
# Generating initial random P and Q
P = np.random.rand(N,K) # matrix size num_users x k_features 
Q = np.random.rand(M,K) # matrix size num_items x k_features
P

array([[0.41067286, 0.01891435, 0.4211022 ],
       [0.15315099, 0.26225727, 0.34055238],
       [0.62798413, 0.93512398, 0.6207347 ],
       [0.90677761, 0.56752736, 0.24975471],
       [0.3758472 , 0.39893089, 0.24137131],
       [0.08558703, 0.70307224, 0.34520321]])

In [10]:
Q

array([[0.99003188, 0.34198183, 0.41385981],
       [0.1622492 , 0.79172852, 0.8654075 ],
       [0.07119679, 0.01047612, 0.82638045],
       [0.63041557, 0.53906891, 0.58856663]])

In [14]:
# Creating a gradient descent process to find optimal P and Q
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    '''
    R: rating matrix
    P: |U| * K (User features matrix)
    Q: |D| * K (Item features matrix)
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter'''
    Q = Q.T #transpose of Q
    for step in range(steps):
        for i in range(len(R)): 
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    # calculate error
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        # calculate gradient with a and beta parameter
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = np.dot(P,Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        # 0.001: local minimum
        if e < 0.001:
            break
    return P, Q.T

In [16]:
# Perform matrix factorization to get the optimum P and Q matrices
nP, nQ = matrix_factorization(R, P, Q, K)
# Predicted R is the dot product of P and the transpose of Q
nR = np.dot(nP, nQ.T)
nR

array([[5.00017199, 2.92635025, 5.24977768, 1.00104532],
       [3.9766888 , 2.15160266, 3.83322088, 0.9986938 ],
       [1.05501736, 0.87049605, 5.32103121, 4.96578725],
       [0.98601173, 0.52442957, 3.67562184, 3.97771972],
       [1.5735104 , 1.12241437, 4.94244536, 4.01327711],
       [1.91403333, 1.11865491, 2.99959296, 1.73161264]])

In [18]:
R # original ratings to compare against predicted

array([[5, 3, 0, 1],
       [4, 0, 0, 1],
       [1, 1, 0, 5],
       [1, 0, 0, 4],
       [0, 1, 5, 4],
       [2, 1, 3, 0]])